In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split


In [ ]:
# Load and resize images from a directory with folder names as labels
def load_and_resize_images_from_folder(folder, target_size=(300, 300)):
    images = []
    labels = []
    class_names = []

    # Check if the folder path is valid
    if not os.path.isdir(folder):
        raise ValueError(f"Invalid folder path: {folder}")

    # Get the list of class names (subfolder names)
    class_names = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]

    # Check if any class subfolders exist
    if not class_names:
        raise ValueError(f"No class subfolders found in {folder}")

    for i, class_name in enumerate(class_names):
        class_path = os.path.join(folder, class_name)
        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                # Resize image to target size
                img = cv2.resize(img, target_size)
                images.append(img)
                labels.append(i)  # Assign the class index as the label

    # Check if any images were loaded
    if not images:
        raise ValueError(f"No images found in {folder}")

    return images, labels, class_names


In [ ]:
#mount google drive to read data
folder = 'RiceLeafDiseaseImages'

import pandas as pd
from google.colab import data_table
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/DSBA6165_Group_Project/'+ folder

In [ ]:
# Define path to the folder containing images
#folder_path = "/DSBA6165/Group Project/RiceLeafDiseaseImages"
#folder_path = "/content/drive/MyDrive/DSBA6165_Group_Project/RiceLeafDiseaseImages"



try:
    # Load and resize images
    images, labels, class_names = load_and_resize_images_from_folder(path)
except ValueError as e:
    print(f"Error: {e}")
    exit()

In [ ]:
# Define Siamese Network for Few-shot Learning
def create_siamese_network(input_shape):
    input_img = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu')(input_img)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu')(x)
    encoded = layers.Dense(128)(x)

    return models.Model(input_img, encoded)


In [ ]:
# Contrastive loss function
def contrastive_loss(y_true, y_pred):
    margin = 1
    y_true = tf.cast(y_true, tf.float32)  # Cast y_true to float32
    return tf.reduce_mean(y_true * tf.square(y_pred) + (1 - y_true) * tf.square(tf.maximum(margin - y_pred, 0)))


In [ ]:
# Pairwise image generator for few-shot learning
def generate_pairs(X, y, batch_size=32, num_pairs=5):
    while True:
        pairs = []
        labels = []
        for _ in range(batch_size):
            idx = np.random.choice(len(X), size=2, replace=False)
            img1, img2 = X[idx[0]], X[idx[1]]
            label = 1 if y[idx[0]] == y[idx[1]] else 0
            pairs.append((img1, img2))
            labels.append(label)
        yield ([np.array(pairs)[:, 0], np.array(pairs)[:, 1]], np.array(labels))

In [ ]:
# Load and preprocess data
try:
    # Load and resize images
    images, labels, class_names = load_and_resize_images_from_folder(path)
except ValueError as e:
    print(f"Error: {e}")
    exit()

In [ ]:
# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Create siamese network
input_shape = X_train[0].shape
siamese_network = create_siamese_network(input_shape)

In [ ]:
# Define two inputs for the siamese network
input_a = layers.Input(shape=input_shape)
input_b = layers.Input(shape=input_shape)

In [ ]:
# Generate the encodings for the two inputs using the siamese network
encoding_a = siamese_network(input_a)
encoding_b = siamese_network(input_b)

In [ ]:
# Calculate the Euclidean distance between the encodings
distance = layers.Lambda(lambda tensors: tf.sqrt(tf.reduce_sum(tf.square(tensors[0] - tensors[1]), axis=-1)))([encoding_a, encoding_b])

# Create the siamese model
siamese_model = models.Model(inputs=[input_a, input_b], outputs=distance)

# Compile the siamese model with contrastive loss
siamese_model.compile(loss=contrastive_loss, optimizer='adam', metrics=['accuracy'])

# Train the siamese model using few-shot learning
batch_size = 32
num_pairs = 5
siamese_model.fit(generate_pairs(X_train, y_train, batch_size, num_pairs),
                  steps_per_epoch=len(X_train) // batch_size,
                  epochs=10)


Epoch 1/10
148/148 [==============================] - 1727s 12s/step - loss: nan - accuracy: 0.2365
Epoch 2/10
148/148 [==============================] - 1710s 12s/step - loss: nan - accuracy: 0.2432
Epoch 3/10
148/148 [==============================] - 1694s 11s/step - loss: nan - accuracy: 0.2027
Epoch 4/10
148/148 [==============================] - 1689s 11s/step - loss: nan - accuracy: 0.2568
Epoch 5/10
148/148 [==============================] - 1687s 11s/step - loss: nan - accuracy: 0.3041
Epoch 6/10
148/148 [==============================] - 1688s 11s/step - loss: nan - accuracy: 0.2432
Epoch 7/10
148/148 [==============================] - 1689s 11s/step - loss: nan - accuracy: 0.2365
Epoch 8/10
148/148 [==============================] - 1687s 11s/step - loss: nan - accuracy: 0.2365
Epoch 9/10
148/148 [==============================] - 1694s 11s/step - loss: nan - accuracy: 0.2500
Epoch 10/10
148/148 [==============================] - 1690s 11s/step - loss: nan - accuracy: 0.2230